<a href="https://colab.research.google.com/github/28421027-byte/chatbot_muhammadrafli_28421027/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# SEL 1: Instalasi & Pembersihan Library (Hapus yang lama dan install yang baru)

# Menghapus library lama/konflik yang mungkin menyebabkan masalah path v1beta
!pip uninstall -y google-genai
!pip uninstall -y google-generativeai
# Menginstal ulang yang terbaru dan Gradio
!pip install -q -U google-generativeai
!pip install --upgrade gradio

Found existing installation: google-genai 1.54.0
Uninstalling google-genai-1.54.0:
  Successfully uninstalled google-genai-1.54.0
Found existing installation: google-generativeai 0.8.5
Uninstalling google-generativeai-0.8.5:
  Successfully uninstalled google-generativeai-0.8.5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.0/23.0 MB 76.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 4.6 MB/s eta 0:00:00
  Attempting uninstall: gradio-client
    Found existing installation: gradio_client 1.14.0
    Uninstalling gradio_client-1.14.0:
      Successfully uninstalled gradio_client-1.14.0
  Attempting uninstall: gradio
    Found existing installation: gradio 5.50.0
    Uninstalling gradio-5.50.0:
      Successfully uninstalled gradio-5.50.0


In [ ]:
# SEL 2: Kode Utama Chatbot (Menggunakan gemini-2.5-flash)

import gradio as gr
import google.generativeai as genai
from google.colab import userdata

# 1. Konfigurasi API Key
try:
    gemini_key = userdata.get('kipli_api_key')
    if not gemini_key:
        raise ValueError("API Key tidak ditemukan. Pastikan sudah disimpan di Secrets.")
    genai.configure(api_key=gemini_key)
except Exception as e:
    print(f"Error Konfigurasi API: {e}")

# 2. Inisialisasi Model
# KODE PERUBAHAN: Menggunakan model 'gemini-2.5-flash' untuk memaksa jalur API modern
model = genai.GenerativeModel('gemini-2.5-flash')

# 3. Fungsi Logika Chatbot (Perbaikan Penanganan Riwayat Final)
def chat_with_gemini(message, history):

    MAX_HISTORY_PAIRS = 7
    recent_history = history[-MAX_HISTORY_PAIRS:]

    full_content = []

    for item in recent_history:
        # Periksa apakah item adalah list/tuple DAN memiliki setidaknya 2 elemen
        if isinstance(item, (list, tuple)) and len(item) >= 2:
            human_text, bot_text = item[:2]

            # Tambahan: Pastikan kedua elemen adalah string (teks)
            if isinstance(human_text, str) and isinstance(bot_text, str):

                # Tambahkan peran 'user'
                full_content.append({"role": "user", "parts": [{"text": human_text}]})

                # Tambahkan peran 'model'
                full_content.append({"role": "model", "parts": [{"text": bot_text}]})

            else:
                print(f"Peringatan: Item riwayat dilewati (bukan teks): {item}")
                continue

        else:
            print(f"Peringatan: Item riwayat dilewati (format dictionary atau tidak cukup elemen): {item}")
            continue

    # 2. Tambahkan pesan pengguna saat ini
    full_content.append({"role": "user", "parts": [{"text": message}]})

    # ... (sisa try-except block)
    try:
        response = model.generate_content(full_content)
        return response.text

    except Exception as e:
        # Jika terjadi error API, kembalikan string error
        return f"Terjadi error saat memproses pesan: {e}"
# 4. Membuat Tampilan (UI) Gradio
demo = gr.ChatInterface(
    fn=chat_with_gemini,
    title="🤖 kipli Chatbot (Colab Edition)",
    description="Chatbot cerdas menggunakan kipli AI",
    examples=["Jelaskan kenapa rama jelek", "Buatkan resep kopi matcha", "Apa tiga tantangan utama dalam adopsi AI?", "apa yang dimaksud dengan gempa bumi?"],
)

# 5. Jalankan Aplikasi
demo.launch(debug=True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://5833f6269bb780b3d0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
